In [1]:
import requests
import json
import pandas as pd
from pandas import json_normalize
import sqlite3
import numpy as np

cmoneyWEB = 'https://m.moneydj.com/jsondata/djjson/FundJsonData.xdjjson?x=XDT003F'

def 每日新聞():
    global df2
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.33'}
    res = requests.get(cmoneyWEB, headers = headers)
    res.encoding = 'utf-8' # requests.models.Response
    # json相關(https://medium.com/datainpoint/scraping-json-with-python-requests-95a56e61096a)
    res_text = res.text # str
    today_json = json.loads(res_text) # dict
    
    # json_normalize()函式被非常廣泛地用於讀取巢狀的json字串並返回一個DataFrame
    df = json_normalize(today_json['ResultSet']['Result'])
    df2 = df.copy()
    df2 = df2.replace('\.[A-Z]*','',regex=True) # 用正則表達式刪除.TW
    df2 = df2.replace('\([^\x00-\xff]*\)','',regex=True) # 正則表達式刪除來源
    df2 = df2.loc[:,('XQID','V3','V4','V1')]
    df2.columns = ['代碼','中文簡稱','新聞內容','日期']
    print(df2)



if __name__ == '__main__':
    每日新聞()

      代碼   中文簡稱                      新聞內容          日期
0   1336     台翰                台翰 拚本季營運加溫  2023/05/08
1   1476     儒鴻              品牌拉貨 儒鴻/聚陽向好  2023/05/08
2   1477     聚陽                四檔消費股 大和按讚  2023/05/08
3   1477     聚陽              品牌拉貨 儒鴻/聚陽向好  2023/05/08
4   1605     華新              華新砸55億 進軍航太鏈  2023/05/08
5   1612     宏泰                 宏泰 專注電線電纜  2023/05/08
6   1722     台肥               台肥賣地 賺進23億元  2023/05/08
7   2002     中鋼                 中鋼虧損 大幅收斂  2023/05/08
8   2313     華通              蘋果PCB鏈 營運將升溫  2023/05/08
9   2317     鴻海              鴻海4月營收 寫同期次高  2023/05/08
10  2353     宏碁        PC產業吹響反攻號角 景氣將邁向復甦  2023/05/08
11  2357     華碩        PC產業吹響反攻號角 景氣將邁向復甦  2023/05/08
12  2409     友達                面板價勁揚 雙虎樂透  2023/05/08
13  2732     六角                六角 全年拚賺一股本  2023/05/08
14  2887    台新金            金控配息1205億 九年低點  2023/05/08
15  2912    統一超                四檔消費股 大和按讚  2023/05/08
16  3260     威剛              威剛跨足新領域 動能帶勁  2023/05/08
17  3376    新日興             

In [ ]:
# 下載財報
import requests

res = requests.get('https://m.moneydj.com/jsondata/djjson/FundJsonData.xdjjson?x=XDT003F')
res.encoding = 'big-5'


lines = res.text.split("XQID")

import pandas as pd
from io import StringIO


dfs = pd.read_csv(StringIO("\n".join(lines).replace(':',''))) # \n為換行的意思,把list轉換為DataFrame，另外把“=”轉換成‘’


dfs.drop(['StatusCode0','Comment"報上有利多個股','FieldLength4','Result[{"'],axis=1,inplace=True)
dfs.columns=['代碼','商品','利多理由']
dfs['代碼'] = dfs['代碼'].apply(lambda x: x.split('.TW"')[0])
dfs['商品'] = dfs['商品'].apply(lambda x: x.split('V3"')[1])
dfs['利多理由'] = dfs['利多理由'].apply(lambda x: x.split('V4"')[1])
dfs['利多理由'] = dfs['利多理由'].apply(lambda x: x.split('}')[0])
dfs = dfs.set_index('代碼')
dfs